In [ ]:
from dotenv import load_dotenv
import os

# Carga la clave desde el archivo .env
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

api_key

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-05-20",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
import json

# Ruta relativa al archivo cursos.json
with open('cursos.json', 'r', encoding='utf-8') as f:
    lista_de_cursos = json.load(f)

# Ahora puedes usar lista_de_cursos como una lista de diccionarios
for curso in lista_de_cursos:
    print(curso["titulo"])


In [ ]:
# Simulando entrada del usuario:
tema_usuario = "programación en java"  # El tema que el usuario quiere aprender
nivel = "minimalista"  # puede ser "minimalista" o "detallado"

In [ ]:
# Prompt base
prompt_base = f"""
Eres un planificador experto en rutas de aprendizaje.

Con la lista de cursos que te doy a continuación, construye un roadmap de aprendizaje.

Tu objetivo es seleccionar los cursos más relevantes que contribuyan al dominio del tema, considerando temas, nivel de dificultad y prerequisitos.
"""

# Parte adaptativa según el nivel
if nivel == "minimalista":
    prompt_base += """
Selecciona solo los cursos estrictamente necesarios para comprender y dominar el tema principal, sin incluir cursos complementarios ni tecnologías adyacentes.
"""
elif nivel == "detallado":
    prompt_base += """
Además de los cursos directamente relacionados con el tema, puedes incluir algunos cursos complementarios que aporten fundamentos técnicos o habilidades transversales útiles en contextos reales.

Evita incluir cursos centrados en tecnologías no esenciales al tema principal.
"""

# Continuación fija del prompt
prompt_base += f"""

Devuélveme exclusivamente una lista de aristas (duplas) en este formato:

[(curso_prerrequisito_id, curso_dependiente_id), ...]

Cada dupla indica que el primer curso debe ser completado antes que el segundo. Ordena el camino desde los más básicos hasta los más avanzados.

NO incluyas explicaciones ni textos adicionales, solo la lista JSON con las duplas.

Aquí está la lista de cursos:
{lista_de_cursos}
"""

In [ ]:
print(prompt_base)

In [ ]:
messages = [
    (
        "system",
        prompt_base,
    ),
    ("human", f"Me gustaría un roadmap sobre {tema_usuario}."),
]
ai_msg = llm.invoke(messages)
ai_msg

In [ ]:
import json
import re

respuesta = ai_msg.content

# Elimina los delimitadores markdown y espacios innecesarios
contenido_limpio = re.sub(r"```json|```", "", respuesta).strip()

# Convierte el string a objeto Python (lista de diccionarios)
json_resultado = json.loads(contenido_limpio)

# Ahora puedes usarlo como un objeto normal
print(json_resultado)


In [ ]:
# Este es tu nuevo formato
aristas = json_resultado

# Extraer nodos únicos
ids_usados = {origen for origen, destino in aristas} | {destino for origen, destino in aristas}

# Crear mapa de id → título para los nodos que aparecen
id_a_titulo = {
    curso["id"]: curso["titulo"]
    for curso in lista_de_cursos
    if curso["id"] in ids_usados
}

id_a_titulo

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Crear grafo dirigido
G = nx.DiGraph()
for origen, destino in aristas:
    G.add_edge(id_a_titulo.get(origen, origen), id_a_titulo.get(destino, destino))

# Dibujar usando spring_layout (más simple)
plt.figure(figsize=(12, 6))
pos = nx.spring_layout(G, k=1.2)  # puedes ajustar k para separar más los nodos
nx.draw(G, pos, with_labels=True, node_color="lightgreen", node_size=3000, font_size=9, arrows=True)

plt.title(f"ROADMAP {nivel.upper()} - {tema_usuario.upper()}", fontsize=16)
plt.tight_layout()
plt.show()
